In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter("ignore")
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error, r2_score
import pickle

import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from keras.callbacks import ModelCheckpoint
from keras.layers import LSTM, Flatten
from tensorflow.keras.models import load_model
from tensorflow.keras import regularizers
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense, LayerNormalization, MultiHeadAttention, GlobalAveragePooling1D
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam

from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split

# Load Dataset

In [ ]:
from keras.callbacks import History
history = History()


In [ ]:
names_TN = ['Name',
         'Maths_1_10', 'Literature_1_10', 'Physics_1_10', 'Chemistry_1_10', 'Biology_1_10', 'History_1_10', 'Geography_1_10', 'English_1_10', 'Civic Education_1_10',
         'Maths_2_10', 'Literature_2_10', 'Physics_2_10', 'Chemistry_2_10', 'Biology_2_10', 'History_2_10', 'Geography_2_10', 'English_2_10', 'Civic Education_2_10',
         'Maths_1_11', 'Literature_1_11', 'Physics_1_11', 'Chemistry_1_11', 'Biology_1_11', 'History_1_11', 'Geography_1_11', 'English_1_11', 'Civic Education_1_11',
         'Maths_2_11', 'Literature_2_11', 'Physics_2_11', 'Chemistry_2_11', 'Biology_2_11', 'History_2_11', 'Geography_2_11', 'English_2_11', 'Civic Education_2_11',
         'Maths_1_12', 'Literature_1_12', 'Physics_1_12', 'Chemistry_1_12', 'Biology_1_12', 'History_1_12', 'Geography_1_12', 'English_1_12', 'Civic Education_1_12',
         'Maths_2_12', 'Literature_2_12', 'Physics_2_12', 'Chemistry_2_12', 'Biology_2_12', 'History_2_12', 'Geography_2_12', 'English_2_12', 'Civic Education_2_12',
         'Maths_TN', 'Literature_TN', 'Physics_TN', 'Chemistry_TN', 'Biology_TN', 'English_TN']

names_XH = ['Name',
         'Maths_1_10', 'Literature_1_10', 'Physics_1_10', 'Chemistry_1_10', 'Biology_1_10', 'History_1_10', 'Geography_1_10', 'English_1_10', 'Civic Education_1_10',
         'Maths_2_10', 'Literature_2_10', 'Physics_2_10', 'Chemistry_2_10', 'Biology_2_10', 'History_2_10', 'Geography_2_10', 'English_2_10', 'Civic Education_2_10',
         'Maths_1_11', 'Literature_1_11', 'Physics_1_11', 'Chemistry_1_11', 'Biology_1_11', 'History_1_11', 'Geography_1_11', 'English_1_11', 'Civic Education_1_11',
         'Maths_2_11', 'Literature_2_11', 'Physics_2_11', 'Chemistry_2_11', 'Biology_2_11', 'History_2_11', 'Geography_2_11', 'English_2_11', 'Civic Education_2_11',
         'Maths_1_12', 'Literature_1_12', 'Physics_1_12', 'Chemistry_1_12', 'Biology_1_12', 'History_1_12', 'Geography_1_12', 'English_1_12', 'Civic Education_1_12',
         'Maths_2_12', 'Literature_2_12', 'Physics_2_12', 'Chemistry_2_12', 'Biology_2_12', 'History_2_12', 'Geography_2_12', 'English_2_12', 'Civic Education_2_12',
         'Maths_TN', 'Literature_TN', 'History_TN', 'Geography_TN', 'Civic Education_TN', 'English_TN']

In [ ]:
TN = "/content/drive/MyDrive/NCKH_Review of Student's Performance/Data_Phat/TN_TN_final_header_removed.xlsx"
XH = "/content/drive/MyDrive/NCKH_Review of Student's Performance/Data_Phat/TN_XH_final_header_removed.xlsx"
df_TN = pd.DataFrame(pd.read_excel(TN, header=None, sheet_name='TN_TN_final', names=names_TN)).drop(columns = 'Name', axis=1)
df_XH = pd.DataFrame(pd.read_excel(XH, header=None, sheet_name='TN_XH_final', names=names_XH)).drop(columns = 'Name', axis=1)

In [ ]:
df_TN.head()

,Maths_1_10,Literature_1_10,Physics_1_10,Chemistry_1_10,Biology_1_10,History_1_10,Geography_1_10,English_1_10,Civic Education_1_10,Maths_2_10,...,History_2_12,Geography_2_12,English_2_12,Civic Education_2_12,Maths_TN,Literature_TN,Physics_TN,Chemistry_TN,Biology_TN,English_TN
0,7.6,6.7,6.9,7.0,6.1,7.1,6.3,6.8,8.4,7.0,...,6.9,8.8,6.9,8.2,8.0,7.25,6.75,5.50,6.25,4.6
1,4.2,5.6,6.2,5.9,6.6,7.3,6.9,6.1,8.3,6.7,...,7.6,7.5,6.5,7.4,8.2,7.50,6.00,6.75,5.00,5.0
2,4.9,5.5,5.4,7.2,5.3,6.4,6.3,5.3,8.6,4.9,...,6.9,7.5,5.1,7.6,6.0,6.00,4.75,6.50,5.25,4.0
3,6.3,5.7,6.9,6.8,6.3,6.4,7.9,6.6,8.4,4.7,...,6.8,7.9,6.5,8.4,8.2,8.00,5.50,7.75,4.75,4.2
4,7.3,7.4,6.2,7.7,6.5,6.9,7.4,6.0,8.9,7.4,...,7.4,8.0,7.6,8.1,7.8,7.75,6.75,7.50,6.00,4.8


In [ ]:
df_TN.shape

(566, 60)

In [ ]:
df_XH.head()

,Maths_1_10,Literature_1_10,Physics_1_10,Chemistry_1_10,Biology_1_10,History_1_10,Geography_1_10,English_1_10,Civic Education_1_10,Maths_2_10,...,History_2_12,Geography_2_12,English_2_12,Civic Education_2_12,Maths_TN,Literature_TN,History_TN,Geography_TN,Civic Education_TN,English_TN
0,5.0,6.1,7.2,6.7,7.5,6.4,7.8,7.0,8.9,5.3,...,8.9,8.1,7.5,8.6,7.8,9.00,6.75,7.25,8.75,4.6
1,4.2,5.8,5.2,6.2,5.6,7.1,6.9,5.7,8.4,5.5,...,7.7,6.8,6.8,8.1,7.0,6.25,5.50,7.75,9.00,3.6
2,5.5,6.6,6.3,5.4,7.0,8.8,7.8,6.2,9.1,4.9,...,8.3,8.0,7.3,9.4,7.8,9.50,8.00,7.25,9.25,6.0
3,4.9,6.2,5.8,5.1,6.5,6.9,7.0,5.6,8.8,5.0,...,7.9,7.3,6.6,8.4,4.6,8.25,6.00,7.00,9.25,5.0
4,5.7,6.3,7.3,6.8,6.9,7.9,6.9,5.2,8.4,5.4,...,8.2,8.5,7.6,8.4,7.4,8.00,8.00,8.25,9.25,5.4


In [ ]:
df_XH.shape

(796, 60)

# Function

In [ ]:
def err(subject, actual, pred): #mae,mape,mse,rmse là các hàm mất mát (loss function), ở đây có tác dụng như công thức để tính toán độ chính xác cho hàm Accuracy bên dưới
#với subject là tên môn, actual là giá trị điểm thực sự (ytest), pred là giá trị điểm dự đoán(kq trả về sau khi model dự đoán - ypred),
#so sánh độ chênh lệnh của điểm dự đoán so với điểm thực sự sẽ tìm được độ chính xác
  mae = mean_absolute_error(actual, pred)
  mape = mean_absolute_percentage_error(actual, pred)
  mse = mean_squared_error(actual, pred)
  rmse = np.sqrt(mse)
  res = {'Subject': subject,'MAE': mae, 'MAPE (%)': mape*100, 'MSE': mse, 'RMSE': rmse, 'Accuracy (%)': 100 - mape*100}
  return res

In [ ]:
def Accuracy_TN(result): #tính toán độ chính xác cho các mô hình học máy với dữ liệu đầu vào nhận từ hàm result bên dưới và tính toán bằng cách sử dụng các loss function ở hàm err bên trên như một công thức
    Accuracy = pd.DataFrame(columns=['Subject', 'MAE', 'MAPE (%)', 'MSE', 'RMSE', 'Accuracy (%)'])
    data = []
    data.append(err('Maths', result['Maths'], result['Maths_pred'])) #gọi hàm err để tính toán độ chính xác với tham số đầu vào là kết quả thực (Math) và kết quả model dự đoán được (Maths_pred)
    data.append(err('Literature', result['Literature'], result['Literature_pred']))
    data.append(err('Physics', result['Physics'], result['Physics_pred']))
    data.append(err('Chemistry', result['Chemistry'], result['Chemistry_pred']))
    data.append(err('Biology', result['Biology'], result['Biology_pred']))
    data.append(err('English', result['English'], result['English_pred']))
    dfs = [pd.DataFrame(d, index=[0]) for d in data]
    Accuracy = pd.concat(dfs, ignore_index=True) #Ghi các giá trị tính toán được vào biến Accuracy

    return Accuracy

In [ ]:
def Accuracy_XH(result): #tính toán độ chính xác cho các mô hình học máy với dữ liệu đầu vào nhận từ hàm result bên dưới và tính toán bằng cách sử dụng các loss function ở hàm err bên trên như một công thức
    Accuracy = pd.DataFrame(columns=['Subject', 'MAE', 'MAPE (%)', 'MSE', 'RMSE', 'Accuracy (%)'])
    data = []
    data.append(err('Maths', result['Maths'], result['Maths_pred'])) #gọi hàm err để tính toán độ chính xác với tham số đầu vào là kết quả thực (Math) và kết quả model dự đoán được (Maths_pred)
    data.append(err('Literature', result['Literature'], result['Literature_pred']))
    data.append(err('History', result['History'], result['History_pred']))
    data.append(err('Geography', result['Geography'], result['Geography_pred']))
    data.append(err('Civic Education', result['Civic Education'], result['Civic Education_pred']))
    data.append(err('English', result['English'], result['English_pred']))
    dfs = [pd.DataFrame(d, index=[0]) for d in data]
    Accuracy = pd.concat(dfs, ignore_index=True) #Ghi các giá trị tính toán được vào biến Accuracy

    return Accuracy

In [ ]:
def result_TN(ytest, ypred): #sử dụng như một nơi để lưu dữ liệu, hiển thị thông tin giữa điểm thực tế - điểm dự đoán được
  result = pd.DataFrame({
    'Maths': pd.Series(ytest[:,0]), #lấy tất cả các hàng (tương ứng với tất cả học sinh), và cột thứ 1 (tương ứng với cột điểm môn toán) sau đó gán tất cả cho Maths để tạo dataframe
    'Maths_pred': pd.Series(ypred[:, 0]),
    'Literature': pd.Series(ytest[:,1]),
    'Literature_pred': pd.Series(ypred[:,1 ]),
    'Physics': pd.Series(ytest[:,2]),
    'Physics_pred': pd.Series(ypred[:,2 ]),
    'Chemistry': pd.Series(ytest[:,3]),
    'Chemistry_pred': pd.Series(ypred[:, 3]),
    'Biology': pd.Series(ytest[:,4]),
    'Biology_pred': pd.Series(ypred[:, 4]),
    'English': pd.Series(ytest[:,5]),
    'English_pred': pd.Series(ypred[:, 5])
  })
  for col in result.columns: result[col] = result[col].apply(lambda x: 10 if x > 10 else x)
  return result

In [ ]:
def result_XH(ytest, ypred): #sử dụng như một nơi để lưu dữ liệu, hiển thị thông tin giữa điểm thực tế - điểm dự đoán được
  result = pd.DataFrame({
    'Maths': pd.Series(ytest[:,0]), #lấy tất cả các hàng (tương ứng với tất cả học sinh), và cột thứ 1 (tương ứng với cột điểm môn toán) sau đó gán tất cả cho Maths để tạo dataframe
    'Maths_pred': pd.Series(ypred[:, 0]),
    'Literature': pd.Series(ytest[:,1]),
    'Literature_pred': pd.Series(ypred[:,1 ]),
    'History': pd.Series(ytest[:,2]),
    'History_pred': pd.Series(ypred[:, 2]),
    'Geography': pd.Series(ytest[:,3]),
    'Geography_pred': pd.Series(ypred[:, 3]),
    'Civic Education': pd.Series(ytest[:, 4]),
    'Civic Education_pred': pd.Series(ypred[:, 4]),
    'English': pd.Series(ytest[:,5]),
    'English_pred': pd.Series(ypred[:, 5])

  })
  for col in result.columns: result[col] = result[col].apply(lambda x: 10 if x > 10 else x)
  return result

# Split Dataset

In [ ]:
split = 0.8 #80%
X_XH = df_XH.drop(columns=['Maths_TN', 'Literature_TN', 'History_TN', 'Geography_TN', 'Civic Education_TN', 'English_TN'], axis = 1) #lấy dữ liệu để dùng làm X, chỉ lấy điểm của 4 học kỳ, df.drop nghĩa là sẽ bỏ các cột được liệt kê
X_TN = df_TN.drop(columns=['Maths_TN', 'Literature_TN', 'Physics_TN', 'Chemistry_TN', 'Biology_TN', 'English_TN'], axis = 1)


#những cột bị drop ở bên trên sẽ được gán riêng như bên dưới và dùng làm Y
Y_TN = df_TN.iloc[:, 54:]
Y_XH = df_XH.iloc[:, 54:]

xtrain_TN, xtest_TN, ytrain_TN, ytest_TN = train_test_split(X_TN, Y_TN, test_size = 0.15, random_state= 42)
xtrain_XH, xtest_XH, ytrain_XH, ytest_XH = train_test_split(X_XH, Y_XH, test_size = 0.15, random_state= 42)

xtrain_TN.head()

,Maths_1_10,Literature_1_10,Physics_1_10,Chemistry_1_10,Biology_1_10,History_1_10,Geography_1_10,English_1_10,Civic Education_1_10,Maths_2_10,...,Civic Education_1_12,Maths_2_12,Literature_2_12,Physics_2_12,Chemistry_2_12,Biology_2_12,History_2_12,Geography_2_12,English_2_12,Civic Education_2_12
528,7.8,6.8,8.6,9.0,7.5,6.5,7.9,6.4,8.1,9.4,...,9.4,9.1,8.4,9.4,8.8,8.8,8.3,8.6,7.4,9.1
390,6.3,5.8,6.8,6.3,4.6,5.8,4.6,6.0,7.1,6.9,...,8.5,7.6,5.5,7.1,8.0,7.4,6.9,8.4,7.0,9.1
559,5.9,7.2,8.9,7.9,7.8,7.4,8.4,7.4,8.4,7.2,...,9.0,8.2,7.6,9.3,8.5,7.9,7.7,8.6,7.1,9.4
352,7.1,9.1,7.2,8.0,8.6,8.2,8.6,8.2,9.6,8.5,...,9.1,7.9,7.8,7.6,7.4,8.5,7.3,8.5,9.2,8.7
101,8.7,7.7,9.2,9.7,9.3,9.1,9.4,9.2,9.8,9.3,...,9.9,9.7,9.2,10.0,9.5,9.3,8.7,9.2,8.8,9.4


In [ ]:
ytrain_TN.head()

,Maths_TN,Literature_TN,Physics_TN,Chemistry_TN,Biology_TN,English_TN
528,9.0,7.25,8.50,6.75,6.00,5.6
390,7.2,6.00,6.50,6.00,5.00,5.4
559,7.6,7.67,6.75,7.75,5.25,6.2
352,7.4,9.00,7.00,7.25,6.00,8.6
101,8.4,8.50,9.00,8.00,6.25,6.8


In [ ]:
xtrain_TN = xtrain_TN.values
ytrain_TN = ytrain_TN.values
xtest_TN = xtest_TN.values
ytest_TN = ytest_TN.values

xtrain_XH = xtrain_XH.values
ytrain_XH = ytrain_XH.values
xtest_XH = xtest_XH.values
ytest_XH = ytest_XH.values

In [ ]:
sc = MinMaxScaler(feature_range = (0,1))
sc1 = MinMaxScaler(feature_range = (0,1))
sc2 = MinMaxScaler(feature_range = (0,1))
sc3 = MinMaxScaler(feature_range = (0,1))
sc4 = MinMaxScaler(feature_range = (0,1))
sc5 = MinMaxScaler(feature_range = (0,1))
sc6 = MinMaxScaler(feature_range = (0,1))
sc7 = MinMaxScaler(feature_range = (0,1))

xtrain_TN = sc.fit_transform(xtrain_TN)
ytrain_TN = sc1.fit_transform(ytrain_TN)
xtest_TN = sc2.fit_transform(xtest_TN)
temp_TN = sc3.fit_transform(ytest_TN)

xtrain_XH = sc4.fit_transform(xtrain_XH)
ytrain_XH = sc5.fit_transform(ytrain_XH)
xtest_XH = sc6.fit_transform(xtest_XH)
temp_XH = sc7.fit_transform(ytest_XH)

# Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression

modelLR_TN = LinearRegression().fit(xtrain_TN, ytrain_TN) #huấn luyện
modelLR_XH = LinearRegression().fit(xtrain_XH, ytrain_XH)

predLR_TN = modelLR_TN.predict(xtest_TN) #dự đoán trên tập kiểm tra (xtest)
predLR_XH = modelLR_XH.predict(xtest_XH)

print("predLR size: ", predLR_TN.shape)
print("ytest size: ", ytest_TN.shape)

predLR size:  (85, 6)
ytest size:  (85, 6)


In [ ]:
predLR_TN

array([[0.56580228, 0.51651616, 0.49659917, 0.63716172, 0.43264339,
        0.46195513],
       [0.5841985 , 0.4546547 , 0.58591169, 0.7490493 , 0.39570384,
        0.55147526],
       [0.67457149, 0.5845403 , 0.55815683, 0.66721805, 0.61986715,
        0.51969349],
       [0.47350677, 0.36897703, 0.38816899, 0.48802224, 0.34842485,
        0.45263917],
       [0.50562244, 0.51185873, 0.48524179, 0.51883911, 0.34528218,
        0.37184282],
       [0.45943329, 0.51935189, 0.47070535, 0.20088852, 0.18802355,
        0.1931901 ],
       [0.51111043, 0.5530055 , 0.70141173, 0.76157633, 0.35857248,
        0.46508338],
       [0.28438649, 0.41253498, 0.36187503, 0.32847422, 0.1699288 ,
        0.27799099],
       [0.68390009, 0.56076022, 0.48685321, 0.72024837, 0.62676683,
        0.59582079],
       [0.45379516, 0.40084629, 0.58560948, 0.57422582, 0.28738825,
        0.81491494],
       [0.56393578, 0.71009846, 0.50792042, 0.60651855, 0.57748476,
        0.45690873],
       [0.71017328, 0

In [ ]:
predLR_TN = sc3.inverse_transform(predLR_TN)
predLR_XH = sc7.inverse_transform(predLR_XH)

In [ ]:
resultLR_TN = result_TN(ytest_TN, predLR_TN) #lưu kq bằng hàm result và hiển thị
resultLR_TN

,Maths,Maths_pred,Literature,Literature_pred,Physics,Physics_pred,Chemistry,Chemistry_pred,Biology,Biology_pred,English,English_pred
0,7.8,7.202690,7.50,7.566065,4.25,6.482996,6.50,6.935809,4.00,5.562182,4.6,6.248904
1,8.2,7.287313,6.00,7.318619,7.75,6.929558,8.75,7.495247,5.75,5.322075,7.6,6.839737
2,8.6,7.703029,8.75,7.838161,5.50,6.790784,8.75,7.086090,7.00,6.779136,6.2,6.629977
3,7.0,6.778131,7.00,6.975908,5.50,5.940845,6.25,6.190111,5.25,5.014762,5.0,6.187419
4,6.8,6.925863,6.50,7.547435,5.75,6.426209,6.25,6.344196,6.00,4.994334,4.4,5.654163
...,...,...,...,...,...,...,...,...,...,...,...,...
80,7.0,6.988358,8.50,8.560248,6.50,6.057699,8.00,6.810132,6.75,5.918998,8.4,7.558358
81,6.8,6.132066,5.50,7.002152,4.50,4.809123,6.00,4.762670,5.00,5.427322,8.2,7.393943
82,7.6,7.969758,8.00,8.587286,8.75,7.925574,7.25,5.801088,6.50,6.421671,8.4,7.695599
83,7.0,6.220446,6.00,7.102755,5.50,5.587514,7.50,4.820877,3.75,4.480063,5.0,3.806314


In [ ]:
resultLR_XH = result_XH(ytest_XH, predLR_XH) #lưu kq bằng hàm result và hiển thị
resultLR_XH

,Maths,Maths_pred,Literature,Literature_pred,History,History_pred,Geography,Geography_pred,Civic Education,Civic Education_pred,English,English_pred
0,6.4,5.680702,6.00,6.995730,4.00,6.009640,7.25,7.618029,8.00,8.650299,4.0,3.933580
1,8.2,7.031804,8.50,8.155446,8.25,7.458609,7.50,8.184169,8.75,9.182257,6.8,6.998803
2,6.6,6.981417,7.00,7.966768,6.25,6.757154,7.25,8.129622,8.50,9.312716,5.4,5.405296
3,3.8,5.201380,6.00,6.652854,4.75,3.161908,7.50,6.739454,7.75,8.652785,6.6,5.516326
4,8.6,6.821353,8.25,7.765723,8.75,8.401630,8.25,8.596504,9.50,9.431273,5.6,5.255914
...,...,...,...,...,...,...,...,...,...,...,...,...
115,7.2,5.646944,6.25,7.336546,4.25,4.833260,5.75,6.937554,8.75,9.084420,4.6,3.681351
116,2.8,4.982206,6.25,7.331714,4.75,3.542570,5.75,6.725838,6.75,8.557471,3.0,3.122701
117,7.6,6.628865,8.75,7.874134,6.00,7.409392,8.50,8.363216,8.00,9.513960,6.0,5.738618
118,5.0,5.318718,6.50,7.402814,4.00,3.845695,6.50,6.920268,8.25,8.836197,2.8,3.123330


In [ ]:
Accuracy_TN(resultLR_TN) #tính toán độ chính xác của model theo dự đoán về từng môn

,Subject,MAE,MAPE (%),MSE,RMSE,Accuracy (%)
0,Maths,0.700967,9.179706,0.689135,0.830142,90.820294
1,Literature,0.638714,8.779076,0.599132,0.774036,91.220924
2,Physics,0.783686,13.188673,0.969541,0.984653,86.811327
3,Chemistry,0.884808,12.211531,1.248203,1.117230,87.788469
4,Biology,0.852601,15.504530,1.173006,1.083054,84.495470
5,English,0.785147,13.704226,0.932655,0.965741,86.295774


In [ ]:
Accuracy_XH(resultLR_XH)

,Subject,MAE,MAPE (%),MSE,RMSE,Accuracy (%)
0,Maths,0.851133,14.794815,1.152018,1.073321,85.205185
1,Literature,0.763310,11.408286,1.017131,1.008529,88.591714
2,History,1.170823,23.348655,2.335817,1.528338,76.651345
3,Geography,0.809737,12.161848,0.989625,0.994799,87.838152
4,Civic Education,0.699469,8.642392,0.759784,0.871656,91.357608
5,English,0.834872,16.770965,1.173895,1.083464,83.229035


# MLP

In [ ]:
class MLPModel(Model):
    def __init__(self, n_class):
        super().__init__()
        self.flatten = Flatten()
        self.fc1 = Dense(128, activation='relu')
        self.fc2 = Dense(62, activation='relu')
        self.fc3 = Dense(32, activation='relu')
        self.fc4 = Dense(n_class, activation='linear')

    def call(self, inputs):
        features = self.flatten(inputs)
        features = self.fc1(features)
        features = self.fc2(features)
        features = self.fc3(features)
        features = self.fc4(features)
        return features

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate = 0.01)
optimizer2 = tf.keras.optimizers.Adam(learning_rate = 0.01)

In [ ]:
MLP_TN = MLPModel(ytrain_TN.shape[1])
MLP_XH = MLPModel(ytrain_XH.shape[1])

MLP_TN.compile(optimizer = optimizer, loss = 'mean_squared_error')
MLP_XH.compile(optimizer = optimizer2, loss = 'mean_squared_error')

In [ ]:
MLP_TN.fit(xtrain_TN, ytrain_TN, epochs = 300, batch_size = 128, callbacks = [history])

Epoch 1/400
4/4 [==============================] - 1s 4ms/step - loss: 5371.2954
Epoch 2/400
4/4 [==============================] - 0s 4ms/step - loss: 0.8501
Epoch 3/400
4/4 [==============================] - 0s 3ms/step - loss: 0.0745
Epoch 4/400
4/4 [==============================] - 0s 3ms/step - loss: 0.0817
Epoch 5/400
4/4 [==============================] - 0s 3ms/step - loss: 0.0453
Epoch 6/400
4/4 [==============================] - 0s 4ms/step - loss: 0.0416
Epoch 7/400
4/4 [==============================] - 0s 3ms/step - loss: 0.0381
Epoch 8/400
4/4 [==============================] - 0s 3ms/step - loss: 0.0285
Epoch 9/400
4/4 [==============================] - 0s 3ms/step - loss: 0.0292
Epoch 10/400
4/4 [==============================] - 0s 4ms/step - loss: 0.0286
Epoch 11/400
4/4 [==============================] - 0s 3ms/step - loss: 0.0273
Epoch 12/400
4/4 [==============================] - 0s 4ms/step - loss: 0.0268
Epoch 13/400
4/4 [==============================] - 0s 5ms

In [ ]:
MLP_XH.fit(xtrain_XH, ytrain_XH, epochs = 400, batch_size = 128, callbacks = [history])

Epoch 1/400
6/6 [==============================] - 1s 3ms/step - loss: 2492.9968
Epoch 2/400
6/6 [==============================] - 0s 3ms/step - loss: 0.2315
Epoch 3/400
6/6 [==============================] - 0s 3ms/step - loss: 0.0576
Epoch 4/400
6/6 [==============================] - 0s 3ms/step - loss: 0.0457
Epoch 5/400
6/6 [==============================] - 0s 3ms/step - loss: 0.0355
Epoch 6/400
6/6 [==============================] - 0s 3ms/step - loss: 0.0310
Epoch 7/400
6/6 [==============================] - 0s 3ms/step - loss: 0.0277
Epoch 8/400
6/6 [==============================] - 0s 3ms/step - loss: 0.0250
Epoch 9/400
6/6 [==============================] - 0s 3ms/step - loss: 0.0245
Epoch 10/400
6/6 [==============================] - 0s 3ms/step - loss: 0.0242
Epoch 11/400
6/6 [==============================] - 0s 3ms/step - loss: 0.0239
Epoch 12/400
6/6 [==============================] - 0s 3ms/step - loss: 0.0239
Epoch 13/400
6/6 [==============================] - 0s 3ms

In [ ]:
predMLP_TN = MLP_TN.predict(xtest_TN)
predMLP_TN.shape

3/3 [==============================] - 0s 3ms/step


(85, 6)

In [ ]:
predMLP_XH = MLP_XH.predict(xtest_XH)

4/4 [==============================] - 0s 3ms/step


In [ ]:
predMLP_TN = sc3.inverse_transform(predMLP_TN)

In [ ]:
predMLP_XH = sc7.inverse_transform(predMLP_XH)

In [ ]:
resultMLP_TN = result_TN(ytest_TN, predMLP_TN)
resultMLP_TN

,Maths,Maths_pred,Literature,Literature_pred,Physics,Physics_pred,Chemistry,Chemistry_pred,Biology,Biology_pred,English,English_pred
0,7.8,7.079370,7.50,7.782996,4.25,6.568300,6.50,6.420755,4.00,5.509668,4.6,6.089921
1,8.2,7.259748,6.00,7.913258,7.75,6.769162,8.75,6.616834,5.75,5.741956,7.6,6.353987
2,8.6,7.443695,8.75,8.046097,5.50,6.973997,8.75,6.816790,7.00,5.978837,6.2,6.623277
3,7.0,6.672214,7.00,7.488965,5.50,6.114910,6.25,5.978162,5.25,4.985343,5.0,5.493864
4,6.8,6.796009,6.50,7.578364,5.75,6.252761,6.25,6.112731,6.00,5.144762,4.4,5.675093
...,...,...,...,...,...,...,...,...,...,...,...,...
80,7.0,7.356777,8.50,7.983328,6.50,6.877209,8.00,6.722308,6.75,5.866907,8.4,6.496033
81,6.8,6.485967,5.50,7.354465,4.50,5.907514,6.00,5.775704,5.00,4.745499,8.2,5.221207
82,7.6,7.743080,8.00,8.262300,8.75,7.307378,7.25,7.142234,6.50,6.364378,8.4,7.061563
83,7.0,5.879661,6.00,6.916615,5.50,5.232358,7.50,5.116627,3.75,3.964712,5.0,4.333601


In [ ]:
resultMLP_XH = result_XH(ytest_XH, predMLP_XH)
resultMLP_XH

,Maths,Maths_pred,Literature,Literature_pred,History,History_pred,Geography,Geography_pred,Civic Education,Civic Education_pred,English,English_pred
0,6.4,6.047203,6.00,7.419131,4.00,5.460053,7.25,7.294791,8.00,8.974473,4.0,4.714330
1,8.2,7.313307,8.50,8.444589,8.25,7.567067,7.50,8.263190,8.75,9.717106,6.8,6.458358
2,6.6,7.021695,7.00,8.208403,6.25,7.081774,7.25,8.040146,8.50,9.546061,5.4,6.056669
3,3.8,5.608619,6.00,7.063910,4.75,4.730176,7.50,6.959334,7.75,8.717222,6.6,4.110192
4,8.6,7.244537,8.25,8.388889,8.75,7.452622,8.25,8.210590,9.50,9.676768,5.6,6.363630
...,...,...,...,...,...,...,...,...,...,...,...,...
115,7.2,5.843846,6.25,7.254426,4.25,5.121633,5.75,7.139251,8.75,8.855194,4.6,4.434211
116,2.8,4.725616,6.25,6.348738,4.75,3.260709,5.75,6.283956,6.75,8.199298,3.0,2.893877
117,7.6,7.058818,8.75,8.238470,6.00,7.143553,8.50,8.068541,8.00,9.567836,6.0,6.107806
118,5.0,5.530781,6.50,7.000866,4.00,4.600639,6.50,6.899799,8.25,8.671566,2.8,4.002972


In [ ]:
Accuracy_TN(resultMLP_TN)

,Subject,MAE,MAPE (%),MSE,RMSE,Accuracy (%)
0,Maths,0.710982,9.256648,0.721422,0.849366,90.743352
1,Literature,0.737573,10.087523,0.756534,0.869790,89.912477
2,Physics,0.839713,14.279086,1.092540,1.045246,85.720914
3,Chemistry,0.972582,13.197604,1.389869,1.178927,86.802396
4,Biology,0.822413,14.568988,1.219292,1.104215,85.431012
5,English,1.067457,17.876464,1.710746,1.307955,82.123536


In [ ]:
Accuracy_XH(resultMLP_XH)

,Subject,MAE,MAPE (%),MSE,RMSE,Accuracy (%)
0,Maths,0.881859,15.720437,1.216347,1.102881,84.279563
1,Literature,0.836684,12.772302,1.251242,1.118589,87.227698
2,History,1.221409,25.324509,2.418387,1.555117,74.675491
3,Geography,0.803104,12.033211,0.978542,0.989213,87.966789
4,Civic Education,0.809523,9.947910,0.966274,0.982993,90.052090
5,English,0.909632,18.691059,1.301383,1.140782,81.308941


# LSTM

In [ ]:
xtrain_timesteps_TN = xtrain_TN.reshape((xtrain_TN.shape[0],1,xtrain_TN.shape[1]))
xtest_timesteps_TN = xtest_TN.reshape((xtest_TN.shape[0],1,xtest_TN.shape[1]))

xtrain_timesteps_XH = xtrain_XH.reshape((xtrain_XH.shape[0],1,xtrain_XH.shape[1]))
xtest_timesteps_XH = xtest_XH.reshape((xtest_XH.shape[0],1,xtest_XH.shape[1]))

In [ ]:
LSTM = tf.keras.layers.LSTM

In [ ]:
class LSTMModel(Model):
  def __init__(self, n_class):
    super().__init__()
    self.LSTM1 = LSTM(128, activation = 'relu', return_sequences = True)
    self.LSTM2 = LSTM(64, activation = 'relu', return_sequences= True)
    self.LSTM3 = LSTM(32, activation = 'relu', return_sequences= False)
    self.fc1 = Dense(n_class, activation = 'linear')

  def call(self, inputs):
    features = self.LSTM1(inputs)
    features = self.LSTM2(features)
    features = self.LSTM3(features)
    features = self.fc1(features)
    return features


In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001)
optimizer2 = tf.keras.optimizers.Adam(learning_rate = 0.001)

In [ ]:
LSTMmodel_TN = LSTMModel(ytrain_TN.shape[1])
LSTMmodel_TN.compile(optimizer = optimizer, loss = 'mean_squared_error')

In [ ]:
LSTMmodel_XH = LSTMModel(ytrain_XH.shape[1])
LSTMmodel_XH.compile(optimizer = optimizer2, loss = 'mean_squared_error')

In [ ]:
LSTMmodel_TN.fit(xtrain_timesteps_TN, ytrain_TN, epochs = 200, batch_size = 128, callbacks = [history])

Epoch 1/200
4/4 [==============================] - 4s 10ms/step - loss: 0.3723
Epoch 2/200
4/4 [==============================] - 0s 9ms/step - loss: 0.3647
Epoch 3/200
4/4 [==============================] - 0s 9ms/step - loss: 0.3556
Epoch 4/200
4/4 [==============================] - 0s 9ms/step - loss: 0.3429
Epoch 5/200
4/4 [==============================] - 0s 14ms/step - loss: 0.3233
Epoch 6/200
4/4 [==============================] - 0s 9ms/step - loss: 0.2901
Epoch 7/200
4/4 [==============================] - 0s 9ms/step - loss: 0.2405
Epoch 8/200
4/4 [==============================] - 0s 9ms/step - loss: 0.2234
Epoch 9/200
4/4 [==============================] - 0s 10ms/step - loss: 0.1914
Epoch 10/200
4/4 [==============================] - 0s 9ms/step - loss: 0.1723
Epoch 11/200
4/4 [==============================] - 0s 9ms/step - loss: 0.1507
Epoch 12/200
4/4 [==============================] - 0s 9ms/step - loss: 0.1326
Epoch 13/200
4/4 [==============================] - 0s 10m

In [ ]:
LSTMmodel_XH.fit(xtrain_timesteps_XH, ytrain_XH, epochs = 200, batch_size = 128, callbacks = [history])

Epoch 1/200
6/6 [==============================] - 4s 9ms/step - loss: 0.4228
Epoch 2/200
6/6 [==============================] - 0s 9ms/step - loss: 0.4042
Epoch 3/200
6/6 [==============================] - 0s 8ms/step - loss: 0.3714
Epoch 4/200
6/6 [==============================] - 0s 8ms/step - loss: 0.2987
Epoch 5/200
6/6 [==============================] - 0s 8ms/step - loss: 0.1616
Epoch 6/200
6/6 [==============================] - 0s 8ms/step - loss: 0.1400
Epoch 7/200
6/6 [==============================] - 0s 8ms/step - loss: 0.0863
Epoch 8/200
6/6 [==============================] - 0s 12ms/step - loss: 0.0798
Epoch 9/200
6/6 [==============================] - 0s 8ms/step - loss: 0.0652
Epoch 10/200
6/6 [==============================] - 0s 8ms/step - loss: 0.0544
Epoch 11/200
6/6 [==============================] - 0s 8ms/step - loss: 0.0480
Epoch 12/200
6/6 [==============================] - 0s 9ms/step - loss: 0.0420
Epoch 13/200
6/6 [==============================] - 0s 9ms/s

In [ ]:
predLSTM_TN = LSTMmodel_TN.predict(xtest_timesteps_TN)
predLSTM_TN.shape

3/3 [==============================] - 0s 4ms/step


(85, 6)

In [ ]:
predLSTM_XH = LSTMmodel_XH.predict(xtest_timesteps_XH)
predLSTM_XH.shape

4/4 [==============================] - 0s 3ms/step


(120, 6)

In [ ]:
predLSTM_TN = sc3.inverse_transform(predLSTM_TN)

In [ ]:
predLSTM_XH = sc7.inverse_transform(predLSTM_XH)

In [ ]:
resultLSTM_TN = result_TN(ytest_TN, predLSTM_TN)
resultLSTM_TN

,Maths,Maths_pred,Literature,Literature_pred,Physics,Physics_pred,Chemistry,Chemistry_pred,Biology,Biology_pred,English,English_pred
0,7.8,7.174124,7.50,7.541452,4.25,6.569974,6.50,6.570737,4.00,5.562975,4.6,5.728251
1,8.2,7.464930,6.00,7.503378,7.75,7.035937,8.75,7.036893,5.75,5.969155,7.6,7.065617
2,8.6,7.604478,8.75,7.678595,5.50,7.156966,8.75,7.147217,7.00,6.327177,6.2,6.684082
3,7.0,6.893241,7.00,7.325243,5.50,6.312027,6.25,6.278316,5.25,5.190285,5.0,5.607487
4,6.8,6.932962,6.50,7.614378,5.75,6.404685,6.25,6.271167,6.00,5.058841,4.4,6.045578
...,...,...,...,...,...,...,...,...,...,...,...,...
80,7.0,7.264022,8.50,8.455841,6.50,6.950877,8.00,6.616920,6.75,5.470782,8.4,7.847111
81,6.8,6.627968,5.50,7.118739,4.50,6.447850,6.00,6.210378,5.00,5.811782,8.2,6.701665
82,7.6,7.794939,8.00,8.576626,8.75,7.323263,7.25,6.819330,6.50,5.957376,8.4,7.783757
83,7.0,6.427303,6.00,7.204479,5.50,5.718088,7.50,5.632398,3.75,4.584299,5.0,4.538000


In [ ]:
resultLSTM_XH = result_XH(ytest_XH, predLSTM_XH)
resultLSTM_XH

,Maths,Maths_pred,Literature,Literature_pred,History,History_pred,Geography,Geography_pred,Civic Education,Civic Education_pred,English,English_pred
0,6.4,6.239257,6.00,7.089292,4.00,5.745525,7.25,7.323647,8.00,8.720351,4.0,4.073649
1,8.2,7.222418,8.50,7.878501,8.25,6.713704,7.50,7.940391,8.75,9.199360,6.8,6.974620
2,6.6,6.792403,7.00,7.628782,6.25,5.755509,7.25,7.719862,8.50,9.215039,5.4,4.905784
3,3.8,5.612178,6.00,6.755870,4.75,3.842180,7.50,6.876829,7.75,8.617084,6.6,4.769794
4,8.6,7.116453,8.25,7.781131,8.75,7.888461,8.25,8.485260,9.50,9.434138,5.6,4.951929
...,...,...,...,...,...,...,...,...,...,...,...,...
115,7.2,5.608909,6.25,7.227640,4.25,4.417315,5.75,7.110136,8.75,8.892651,4.6,3.906117
116,2.8,4.972191,6.25,6.955768,4.75,3.808070,5.75,6.684181,6.75,8.558945,3.0,3.428729
117,7.6,7.057466,8.75,7.758699,6.00,6.611519,8.50,8.096869,8.00,9.422350,6.0,5.662035
118,5.0,4.945545,6.50,6.877642,4.00,3.822134,6.50,6.710408,8.25,8.568039,2.8,3.384751


In [ ]:
Accuracy_TN(resultLSTM_TN)

,Subject,MAE,MAPE (%),MSE,RMSE,Accuracy (%)
0,Maths,0.624560,8.213108,0.580759,0.762075,91.786892
1,Literature,0.648898,9.031448,0.622572,0.789032,90.968552
2,Physics,0.855870,14.996777,1.205166,1.097800,85.003223
3,Chemistry,0.837154,11.676511,1.100355,1.048978,88.323489
4,Biology,0.815825,15.045030,1.095207,1.046521,84.954970
5,English,0.786164,13.574692,0.950429,0.974900,86.425308


In [ ]:
Accuracy_XH(resultLSTM_XH)

,Subject,MAE,MAPE (%),MSE,RMSE,Accuracy (%)
0,Maths,0.829938,14.574102,1.109165,1.053169,85.425898
1,Literature,0.729014,10.911502,0.965189,0.982441,89.088498
2,History,1.061894,20.745253,2.028299,1.424184,79.254747
3,Geography,0.769657,11.488844,0.888900,0.942815,88.511156
4,Civic Education,0.667946,8.247422,0.703005,0.838454,91.752578
5,English,0.867867,17.235936,1.278794,1.130838,82.764064


# Save Trained

In [ ]:
import pickle
import torch

In [ ]:
with open('LR_TN_TN.pkl', 'wb') as file:
    pickle.dump(modelLR_TN, file)

In [ ]:
with open('LR_TN_XH.pkl', 'wb') as file:
    pickle.dump(modelLR_XH, file)

In [ ]:
with open('MLP_TN_TN.pkl', 'wb') as file:
    pickle.dump(MLP_TN, file)

In [ ]:
with open('MLP_TN_XH.pkl', 'wb') as file:
    pickle.dump(MLP_XH, file)

In [ ]:
with open('LSTM_TN_TN.pkl', 'wb') as file:
    pickle.dump(LSTMmodel_TN, file)

In [ ]:
with open('LSTM_TN_XH.pkl', 'wb') as file:
    pickle.dump(LSTMmodel_XH, file)